In [1]:
import gzip
import math
import pickle
import zlib
import io
from collections import defaultdict
import pandas as pd
import numpy as np

# import scipy.stats

from sklearn.preprocessing import LabelEncoder

import engines
from utils import *

np.random.seed(2016)

transformers = {}

In [2]:
train_cols = ['customerID', 'Gender', 'State', 'transactionDate','product_code']

In [3]:
df_train = pd.read_csv('products.csv',usecols=train_cols)

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
## fill missing values
#products['customerID'].fillna('BBID_0000', inplace=True)
#df_train['promotion_description'].fillna('no_promo', inplace=True)
df_train['Gender'].fillna('no_gender', inplace=True)
df_train['State'].fillna('no_state', inplace=True)
#df_train['PinCode'].fillna(-1, inplace=True)
#df_train['DOB'].fillna("1", inplace=True)

In [6]:
df_train = df_train[np.isfinite(df_train['product_code'])]

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7981248 entries, 0 to 7981261
Data columns (total 5 columns):
customerID         object
Gender             object
State              object
transactionDate    object
product_code       float64
dtypes: float64(1), object(4)
memory usage: 365.4+ MB


In [8]:
df_train_may_june = df_train[df_train['transactionDate'].isin(['2016-05-28', '2016-06-28'])]

In [9]:
may_june_cols = df_train_may_june['product_code'].astype('int64').head(1000).tolist()

In [10]:
may_june_cols

[1000256435,
 300785148,
 1000256370,
 1000340950003,
 1000436295009,
 1000494471018,
 109006305,
 300781535,
 300939869,
 300987838,
 300650065,
 1000345907,
 300970616,
 1000100137,
 1000487401004,
 1000163797,
 1000310765,
 1000454276007,
 300516394065,
 300516394015,
 300654274,
 1000493507001,
 301025353001,
 300160179,
 1000508508003,
 1000223448003,
 1000234024008,
 300653567,
 1000407614003,
 1000052590,
 1000205140,
 1000196999,
 1000147158,
 300776411,
 108034290,
 1000494206,
 1000117306,
 108027349,
 1000367385001,
 1000470265005,
 1000469365002,
 1000477146005,
 1000459676005,
 1000473959003,
 1000470366012,
 1000450218002,
 300785148,
 1000448227001,
 830013484009,
 1000031866,
 108100333,
 108100115,
 300785150,
 300785148,
 300471165,
 1000446430,
 108100356,
 108100207,
 300577982,
 108100306,
 108100278,
 108100330,
 300087125,
 1000051471,
 1000446427,
 1000446474,
 1000063263,
 108034674,
 300644883,
 300012558,
 300745449,
 1000327844,
 300595508,
 1000096658,
 108

In [11]:
df_train = df_train[df_train['product_code'].isin(may_june_cols) == True]

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1737525 entries, 0 to 7981254
Data columns (total 5 columns):
customerID         object
Gender             object
State              object
transactionDate    object
product_code       float64
dtypes: float64(1), object(4)
memory usage: 79.5+ MB


In [13]:
df_train.reset_index(drop=True)

,customerID,Gender,State,transactionDate,product_code
0,BBID_2041,male,no_state,2016-01-19,3.007851e+08
1,BBID_2041,male,no_state,2016-01-19,1.000010e+09
2,BBID_2041,male,no_state,2016-05-01,1.081003e+08
3,BBID_2041,male,no_state,2016-03-20,1.081003e+08
4,BBID_2041,male,no_state,2016-03-20,1.081003e+08
5,BBID_2041,male,no_state,2016-03-20,1.081003e+08
6,BBID_2042,female,no_state,2016-10-02,3.003613e+08
7,BBID_2042,female,no_state,2015-11-29,3.007851e+08
8,BBID_2043,male,no_state,2016-06-08,1.080204e+08
9,BBID_2043,male,no_state,2017-03-27,3.006151e+08


In [14]:
df_train['product_code'] = df_train['product_code'].astype(int)

In [15]:
df_train['product_code'].value_counts()

300776411     60994
300776410     58437
108037568     51338
300785148     41500
108100382     35259
300785150     34794
300840018     31149
108100362     27963
108005676     27727
300111517     25360
108100306     24668
300776409     22528
108100308     21204
108100294     20174
108100290     19628
108100352     17676
300070102     16987
300785147     16491
108100328     15419
500096181     14785
108001125     14176
108100331     13902
108018729     12792
300326493     12629
108100296     12224
108100267     12093
108100288     11411
108100361     11370
108015133     11129
108015124     10775
              ...  
300554812        32
108036722        32
300681647        28
108018937        28
1000184760       25
300253664        24
830009376        22
1000147079       22
300253663        20
109001376        19
109001126        17
1000154187       17
1000223299       16
300844708        16
1000170133       15
300766165        15
300956136        12
1000049633       12
1000349091       11


In [16]:
df_train = pd.get_dummies(df_train, columns=['product_code'],prefix='_')

df_train.columns=df_train.columns.str.replace('_','')
df_train.reset_index(drop=True)
df_train.head()

,customerID,Gender,State,transactionDate,-2147483648,102107203,108000197,108000217,108000372,108000441,108000523,108000526,108000534,108000537,108000569,108000589,108000613,108000614,108000627,108000643,108000653,108000655,108000659,108000704,108000706,108000969,108000980,108001018,108001038,108001065,108001066,108001107,108001125,108001127,108001134,108001138,108001153,108001172,108001179,108001180,108001199,108001261,108001480,108002296,108002297,108002313,108002326,108002329,108003357,108003451,108003587,108003638,108003639,108004050,108004562,108004563,108004628,108004977,108005676,108005943,108007442,108008174,108008255,108008545,108008585,108008700,108008721,108008727,108009867,108009886,108009896,108009899,108009900,108010118,108010536,108010550,108011717,108013064,108013796,108014145,108014204,108014208,108014548,108014561,108014700,108015124,108015133,108015135,108015361,108015910,108016183,108016189,108016191,108016194,108017210,108017243,108017247,108017697,108018123,108018135,108018273,108018729,108018749,108018937,108019092,108019094,108020341,108020344,108020370,108020371,108020386,108020392,108020412,108020414,108020418,108020428,108020460,108020474,108020477,108020486,108020496,108020516,108020517,108020532,108020603,108021733,108022529,108023376,108023919,108024347,108024722,108024751,108024756,108024797,108024836,108024881,108025238,108026134,108026136,108026387,108026421,108026996,108027005,108027349,108027409,108027413,108027465,108027466,108028589,108028599,108028645,108029656,108030063,108030066,108031973,108031986,108033786,108034290,108034670,108034674,108035034,108035252,108035254,108035440,108035720,108036717,108036722,108037489,108037498,108037568,108037803,108037996,108038304,108042069,108042082,108043278,108100059,108100115,108100116,108100127,108100137,108100145,108100162,108100183,108100186,108100189,108100193,108100194,108100195,108100207,108100213,108100225,108100232,108100243,108100246,108100250,108100253,108100264,108100267,108100275,108100278,108100288,108100290,108100294,108100296,108100297,108100298,108100306,108100308,108100312,108100319,108100328,108100330,108100331,108100333,108100334,108100335,108100336,108100346,108100352,108100355,108100356,108100361,108100362,108100364,108100373,108100379,108100382,109001123,109001125,109001126,109001376,109006305,208001927,208001932,208001941,208001961,208001962,208002292,300004737,300012558,300014687,300024993,300027193,300027793,300027850,300027872,300028156,300033479,300042984,300043603,300047282,300047378,300053834,300063844,300066479,300070102,300081480,300082556,300083325,300087125,300107879,300107894,300111517,300138428,300138647,300149677,300150379,300151449,300160179,300162906,300165590,300171566,300176107,300183684,300183957,300220775,300220782,300220784,300220789,300228572,300229349,300233592,300234140,300234144,300246974,300253663,300253664,300261346,300263114,300286077,300289771,300291250,300309412,300312161,300313472,300317078,300321570,300322106,300326493,300361324,300373477,300373480,300374464,300392495,300392496,300392920,300403095,300403287,300403290,300403293,300403296,300412432,300413862,300419840,300420036,300442567,300444033,300449774,300455123,300455896,300468000,300468206,300468924,300471165,300473128,300485392,300486090,300491428,300497842,300498333,300502013,300506518,300511394,300513427,300513428,300522764,300523228,300523231,300528028,300531329,300535432,300535524,300535676,300537091,300542865,300543027,300553957,300554812,300555507,300556301,300564429,300569686,300577982,300577985,300595508,300599199,300603007,300615072,300615106,300615689,300616374,300616591,300622785,300624169,300628869,300637344,300637812,300642919,300644883,300648529,300650065,300651205,300652624,300653567,300653598,300654249,300654274,300663132,300663254,300669679,300670422,300679276,300679526,300679538,300680079,300680423,300681647,300688276,300691080,300692726,300692768,300692782,300699169,300702014,300702048,300705561,300705578,300706641,3007119

In [17]:
df_train = df_train.rename(columns={'-2147483648': '2147483648'})

In [18]:
df_train.head()

,customerID,Gender,State,transactionDate,2147483648,102107203,108000197,108000217,108000372,108000441,108000523,108000526,108000534,108000537,108000569,108000589,108000613,108000614,108000627,108000643,108000653,108000655,108000659,108000704,108000706,108000969,108000980,108001018,108001038,108001065,108001066,108001107,108001125,108001127,108001134,108001138,108001153,108001172,108001179,108001180,108001199,108001261,108001480,108002296,108002297,108002313,108002326,108002329,108003357,108003451,108003587,108003638,108003639,108004050,108004562,108004563,108004628,108004977,108005676,108005943,108007442,108008174,108008255,108008545,108008585,108008700,108008721,108008727,108009867,108009886,108009896,108009899,108009900,108010118,108010536,108010550,108011717,108013064,108013796,108014145,108014204,108014208,108014548,108014561,108014700,108015124,108015133,108015135,108015361,108015910,108016183,108016189,108016191,108016194,108017210,108017243,108017247,108017697,108018123,108018135,108018273,108018729,108018749,108018937,108019092,108019094,108020341,108020344,108020370,108020371,108020386,108020392,108020412,108020414,108020418,108020428,108020460,108020474,108020477,108020486,108020496,108020516,108020517,108020532,108020603,108021733,108022529,108023376,108023919,108024347,108024722,108024751,108024756,108024797,108024836,108024881,108025238,108026134,108026136,108026387,108026421,108026996,108027005,108027349,108027409,108027413,108027465,108027466,108028589,108028599,108028645,108029656,108030063,108030066,108031973,108031986,108033786,108034290,108034670,108034674,108035034,108035252,108035254,108035440,108035720,108036717,108036722,108037489,108037498,108037568,108037803,108037996,108038304,108042069,108042082,108043278,108100059,108100115,108100116,108100127,108100137,108100145,108100162,108100183,108100186,108100189,108100193,108100194,108100195,108100207,108100213,108100225,108100232,108100243,108100246,108100250,108100253,108100264,108100267,108100275,108100278,108100288,108100290,108100294,108100296,108100297,108100298,108100306,108100308,108100312,108100319,108100328,108100330,108100331,108100333,108100334,108100335,108100336,108100346,108100352,108100355,108100356,108100361,108100362,108100364,108100373,108100379,108100382,109001123,109001125,109001126,109001376,109006305,208001927,208001932,208001941,208001961,208001962,208002292,300004737,300012558,300014687,300024993,300027193,300027793,300027850,300027872,300028156,300033479,300042984,300043603,300047282,300047378,300053834,300063844,300066479,300070102,300081480,300082556,300083325,300087125,300107879,300107894,300111517,300138428,300138647,300149677,300150379,300151449,300160179,300162906,300165590,300171566,300176107,300183684,300183957,300220775,300220782,300220784,300220789,300228572,300229349,300233592,300234140,300234144,300246974,300253663,300253664,300261346,300263114,300286077,300289771,300291250,300309412,300312161,300313472,300317078,300321570,300322106,300326493,300361324,300373477,300373480,300374464,300392495,300392496,300392920,300403095,300403287,300403290,300403293,300403296,300412432,300413862,300419840,300420036,300442567,300444033,300449774,300455123,300455896,300468000,300468206,300468924,300471165,300473128,300485392,300486090,300491428,300497842,300498333,300502013,300506518,300511394,300513427,300513428,300522764,300523228,300523231,300528028,300531329,300535432,300535524,300535676,300537091,300542865,300543027,300553957,300554812,300555507,300556301,300564429,300569686,300577982,300577985,300595508,300599199,300603007,300615072,300615106,300615689,300616374,300616591,300622785,300624169,300628869,300637344,300637812,300642919,300644883,300648529,300650065,300651205,300652624,300653567,300653598,300654249,300654274,300663132,300663254,300669679,300670422,300679276,300679526,300679538,300680079,300680423,300681647,300688276,300691080,300692726,300692768,300692782,300699169,300702014,300702048,300705561,300705578,300706641,30071196

In [19]:
state_dict = {'MADHY PRADESH':'MADHYA PRADESH', 'TAMILNADU':'TAMIL NADU', 'MADHYA  PRADESH':'MADHYA PRADESH', 'HARAYANA':'HARYANA',
             'Jharkhand':'JHARKHAND','Tamilnadu':'TAMIL NADU','Tamil Nadu':'TAMIL NADU','Madhya Pradesh':'MADHYA PRADESH',
             'REST OF WEST BENGAL':'WEST BENGAL', 'west bengal':'WEST BENGAL','Uttar Pradesh':'UTTAR PRADESH', 'Delhi':'DELHI',
             'Bhopal':'BHOPAL','CHHATISGARH':'CHHATTISGARH','CHATTISGARH':'CHHATTISGARH', 'jharkhand':'JHARKHAND','Chandigarh':'CHANDIGARH',
             'UTTAR PRADESH WEST': 'UTTAR PRADESH','ODISHA':'ORISSA','MAHARASTRA':'MAHARASHTRA','madhya pradesh':'MADHYA PRADESH',
             'KARNATAK':'KARNATAKA','JAMMU and KASHMIR':'JAMMU AND KASHMIR','JAMMU KASHMIR':'JAMMU AND KASHMIR','Rajasthan':'RAJASTHAN',
             'east singhbhum':'JHARKHAND', 'ORRISA':'ORISSA','Andhra Pradesh':'ANDHRA PRADESH', 'UTTARANCHAL':'UTTARAKHAND',
             'Uttar pradesh':'UTTAR PRADESH','Maharashtra':'MAHARASHTRA','MP':'MADHYA PRADESH', 'UTTAR PRADESH EAST':'UTTAR PRADESH',
             'Punjab':'PUNJAB','maharashtra':'MAHARASHTRA','Karnataka':'KARNATAKA','M.P.':'MADHYA PRADESH','DAMAN':'DAMAN AND DIU',
             'HUBLI':'KARNATAKA','Tamil nadu':'TAMIL NADU','GUJRAT':'GUJARAT', 'Mp':'MADHYA PRADESH','Madhya pradesh':'MADHYA PRADESH',
             'West Bengal':'WEST BENGAL','Gujarat':'GUJARAT','UP':'UTTAR PRADESH','Chennai':'CHENNAI', 'm.p.':'MADHYA PRADESH',
             'kerala':'KERALA'}

df_train.replace({"State": state_dict}, inplace=True)

In [20]:
df_train_june = df_train[df_train['transactionDate']=='2016-06-28']
df_train_may = df_train[df_train['transactionDate']=='2016-05-28']
df_train_april = df_train[df_train['transactionDate']=='2016-04-28']
df_train_march = df_train[df_train['transactionDate']=='2016-03-28']
df_train_feb = df_train[df_train['transactionDate']=='2016-02-28']
df_train_jan = df_train[df_train['transactionDate']=='2016-01-28']

df_train.drop('transactionDate', axis=1, inplace=True)

In [21]:
dfm = pd.merge(df_train_june,df_train_may, how='left', on=['customerID'], suffixes=('', '_prev'))
dfm = pd.merge(dfm,df_train_april, how='left', on=['customerID'], suffixes=('', '_prev'))
dfm = pd.merge(dfm,df_train_march, how='left', on=['customerID'], suffixes=('', '_prev'))
dfm = pd.merge(dfm,df_train_feb, how='left', on=['customerID'], suffixes=('', '_prev'))
dfm = pd.merge(dfm,df_train_jan, how='left', on=['customerID'], suffixes=('', '_prev'))

In [24]:
dfm.head()

customerID Gender     State transactionDate  2147483648  102107203  \
0  BBID_204735   male  no_state      2016-06-28           0          0   
1  BBID_204735   male  no_state      2016-06-28           0          0   
2  BBID_204735   male  no_state      2016-06-28           0          0   
3  BBID_204735   male  no_state      2016-06-28           0          0   
4  BBID_204735   male  no_state      2016-06-28           0          0   

   108000197  108000217  108000372  108000441  108000523  108000526  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   108000534  108000537  108000569  108000589  108000613  108000614  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   108000627  108000643  108000653  108000655  108000659  108000704  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   108000706  108000969  108000980  108001018  108001038  108001065  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   108001066  108001107  108001125  108001127  108001134  108001138  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   108001153  108001172  108001179  108001180  108001199  108001261  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   108001480  108002296  108002297  108002313  108002326  108002329  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   108003357  108003451  108003587  108003638  108003639  108004050  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   108004562  108004563  108004628  108004977  108005676  108005943  \
0          0          0          0          0          0          0   
1

In [28]:
#prevcols = [col for col in dfm.columns if '_prev' in col and col not in ('customerID','Gender', 'State', 'transactionDate')]
#currcols = [col for col in dfm.columns if '_prev' not in col and col not in ('customerID','Gender', 'State', 'transactionDate')]
prevcols = [col for col in dfm.columns if '_prev' in col]
currcols = [col for col in dfm.columns if '' in col and '_prev' not in col]

In [30]:
prevcols.remove('Gender_prev')
prevcols.remove('State_prev')
prevcols.remove('transactionDate_prev')
#prevcols

In [43]:
currcols.remove('customerID')
currcols.remove('State')
currcols.remove('Gender')
currcols.remove('transactionDate')

In [44]:
for col in prevcols:
    dfm[col].fillna(0, inplace=True)

In [45]:
for col in currcols:
    dfm[col].fillna(0, inplace=True)

In [46]:
dfm.fillna(0, inplace=True)

In [57]:
currcols

['2147483648',
 '102107203',
 '108000197',
 '108000217',
 '108000372',
 '108000441',
 '108000523',
 '108000526',
 '108000534',
 '108000537',
 '108000569',
 '108000589',
 '108000613',
 '108000614',
 '108000627',
 '108000643',
 '108000653',
 '108000655',
 '108000659',
 '108000704',
 '108000706',
 '108000969',
 '108000980',
 '108001018',
 '108001038',
 '108001065',
 '108001066',
 '108001107',
 '108001125',
 '108001127',
 '108001134',
 '108001138',
 '108001153',
 '108001172',
 '108001179',
 '108001180',
 '108001199',
 '108001261',
 '108001480',
 '108002296',
 '108002297',
 '108002313',
 '108002326',
 '108002329',
 '108003357',
 '108003451',
 '108003587',
 '108003638',
 '108003639',
 '108004050',
 '108004562',
 '108004563',
 '108004628',
 '108004977',
 '108005676',
 '108005943',
 '108007442',
 '108008174',
 '108008255',
 '108008545',
 '108008585',
 '108008700',
 '108008721',
 '108008727',
 '108009867',
 '108009886',
 '108009896',
 '108009899',
 '108009900',
 '108010118',
 '108010536',
 '108

In [53]:
dfm.columns

Index(['customerID', 'Gender', 'State', 'transactionDate', '2147483648',
       '102107203', '108000197', '108000217', '108000372', '108000441',
       ...
       '1000525825_prev', '1000526185_prev', '1000526240_prev',
       '1000531372_prev', '1000531403_prev', '1000533532_prev',
       '1000535711_prev', '1000535812_prev', '1000540911_prev',
       '1000543204_prev'],
      dtype='object', length=4027)

In [58]:
for col in currcols:
    dfm[col] = dfm[col] - dfm[col+'_prev']
    dfm[col] = dfm[col].apply(lambda x: max(x,0))

TypeError: unorderable types: str() < int()

In [ ]:
prevcols2 = [col for col in dfm.columns if '_prev' in col and col not in prevcols]
dfm.drop(prevcols2, axis=1, inplace=True)

In [ ]:
dfm = dfm[dfm[currcols].sum(axis=1) >0]

In [ ]:
dfm[currcols].sum().sum()

In [ ]:
dfm.head()

In [ ]:
df_new = pd.DataFrame()

for index, row in dfm.iterrows():
    if index%1000 ==0:
        print(index)
    for i,col in enumerate(currcols):
        if row[col] == 1:
            row['target'] = currcols.index(col)
            df_new = df_new.append(row)
            
df_new.drop(currcols, axis=1, inplace=True)

In [ ]:
df_new.to_csv('cleaned_data/juneExtraMulticlass_1000prods_may_june_only.csv')